# U-Net: Convolutional networks for biomedical image segmentation (O. Ronneberger et al., 2015)

In [1]:
!pip list | grep torch

pytorch-lightning        1.5.0
torch                    1.11.0
torchmetrics             0.5.1
torchvision              0.12.0

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from pathlib import Path
import pytorch_lightning as pl
from fastmri.data.subsample import create_mask_for_mask_type
from fastmri.data.transforms import UnetDataTransform
from fastmri.pl_modules import FastMriDataModule, UnetModule

/home/lrobaina/school/IMPACT-MRI/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## K-Space Mask for transforming the input data

In [3]:
mask_types = [
    "random",
    "equispaced",
    "equispaced_fraction",
    "magic",
    "magic_fraction"
]
mask_type = mask_types[0]

In [4]:
# Number of center lines to use in mask
center_fractions = [0.09]

In [5]:
# acceleration rates to use for masks
accelerations = [4]

In [6]:
mask = create_mask_for_mask_type(
    mask_type, center_fractions, accelerations
)
type(mask)

fastmri.data.subsample.RandomMaskFunc

### use random masks for train transform, fixed masks for val transform

In [7]:
# Data specific Parameters
data_path = Path('../data/singlecoil_datasets/')
test_path = Path('../data/singlecoil_datasets/singlecoil_test/')
challenge = "singlecoil"
test_split = "test"
# Fraction of slices in the dataset to use (train split only). 
# If not given all will be used. Cannot set together with volume_sample_rate.
sample_rate = None
val_sample_rate = None
test_sample_rate = None
volume_sample_rate = None
val_volume_sample_rate = None
test_volume_sample_rate = None
use_dataset_cache_file = True
combine_train_val = False

# data loader arguments
batch_size = 1
num_workers = 4

In [8]:
train_transform = UnetDataTransform(challenge, mask_func=mask, use_seed=False)
train_transform

In [9]:
val_transform = UnetDataTransform(challenge, mask_func=mask)

In [10]:
test_transform = UnetDataTransform(challenge)

In [11]:
data_module = FastMriDataModule(
        data_path=data_path,
        challenge=challenge,
        train_transform=train_transform,
        val_transform=val_transform,
        test_transform=test_transform,
        test_split=test_split,
        test_path=test_path,
        sample_rate=sample_rate,
        batch_size=batch_size,
        num_workers=num_workers,
        distributed_sampler=False,
)
data_module.challenge

'singlecoil'

## UNet Model

In [12]:
##############################
# UNet Model Hyperparameters #
##############################
in_chans=1          # number of input channels to U-Net
out_chans=1         # number of output chanenls to U-Net
chans=32            # number of top-level U-Net channels
num_pool_layers=4   # number of U-Net pooling layers
drop_prob=0.0       # dropout probability
lr=0.001            # RMSProp learning rate
lr_step_size=40     # epoch at which to decrease learning rate
lr_gamma=0.1        # extent to which to decrease learning rate
weight_decay=0.0    # weight decay regularization strength

In [13]:
model = UnetModule(
        in_chans=in_chans,
        out_chans=out_chans,
        chans=chans,
        num_pool_layers=num_pool_layers,
        drop_prob=drop_prob,
        lr=lr,
        lr_step_size=lr_step_size,
        lr_gamma=lr_gamma,
        weight_decay=weight_decay,
)

## Trainer

In [16]:
trainer_config = dict(
    #gpus=1,                     # number of gpus to use
    #replace_sampler_ddp=False,  # this is necessary for volume dispatch during val
    strategy=None,               # what distributed version to use
    #seed=42,                    # random seed
    deterministic=True,         # makes things slower, but deterministic
    default_root_dir='../logs',  # directory for logs and checkpoints
    max_epochs=50,              # max number of epochs
)

In [17]:
trainer = pl.Trainer(**trainer_config)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


## Run Training

In [18]:
trainer.fit(model, datamodule=data_module)


  | Name             | Type                 | Params
----------------------------------------------------------
0 | NMSE             | DistributedMetricSum | 0     
1 | SSIM             | DistributedMetricSum | 0     
2 | PSNR             | DistributedMetricSum | 0     
3 | ValLoss          | DistributedMetricSum | 0     
4 | TotExamples      | DistributedMetricSum | 0     
5 | TotSliceExamples | DistributedMetricSum | 0     
6 | unet             | Unet                 | 7.8 M 
----------------------------------------------------------
7.8 M     Trainable params
0         Non-trainable params
7.8 M     Total params
31.024    Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/home/lrobaina/school/IMPACT-MRI/.venv/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:56: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/home/lrobaina/school/IMPACT-MRI/fastmri/pl_modules/unet_module.py:104: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1, 320, 320])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  "val_loss": F.l1_loss(output, batch.target),


ValueError: Input images must have the same dimensions.